In [1]:
import gensim.downloader

In [2]:
glove_vectors = gensim.downloader.load('glove-twitter-50')

In [27]:
glove_vectors.similarity('the' , 'there')

0.8969455

In [41]:
from flask import Flask, request, render_template

app = Flask(__name__)


@app.route('/')
def my_form():
    return render_template('my-form.html')


@app.route('/', methods=['POST'])

def my_form_post():
    text = request.form['text']
    number = request.form['number']

    list_of_words = text.split()

    hashtags_list = []
    most_similar_vec = []

    for word in list_of_words:
        hashtags_list.append(word.lower())
        if glove_vectors.__contains__(word):

            most_similar_vec = glove_vectors.most_similar(word)
            count = 0

            for word_1, similarity in most_similar_vec:

                hashtags_list.append(word_1)
                count = count + 1

    # finding the cosine similarity of the generated words with the entire entered text and sorting it in decending order w.r.t similarity

    final_list = []
    for hash_word in hashtags_list:
        cosine_sim = 0
        # not including words with length less or equal to 4
        if len(hash_word) <= 3:
            continue
        for word in text:

            # checking if the input word is in the vocabulary
            if glove_vectors.__contains__(word) and glove_vectors.__contains__(hash_word):
                # comparing each word of the generated word with the entire text
                cosine_sim = cosine_sim + \
                    glove_vectors.similarity(word, hash_word)

        final_list.append((hash_word, cosine_sim))

    # sorting with respect to total cosine similarity in increasing order 
    # words having high total cosine similarity are not good hashtags 

    final_list.sort(key=lambda x: x[1])


    count = 0
    # removing duplicates
    newlist = sorted(set(final_list), key=lambda x: final_list.index(x))

    processed_text = ""

    ct = 0

    for word, similarity in newlist:

        if glove_vectors.__contains__(word):
            if glove_vectors.similarity(word , 'the') >= 0.8 : #filtering some common words related to 'the'
                continue
        processed_text = processed_text+'#'+word + ' '

        ct = ct + 1

        if str(ct) == number:
            break

    return render_template('result.html' , result = processed_text)


app.run(host='localhost', port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Feb/2022 14:06:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 14:07:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2022 14:07:12] "POST / HTTP/1.1" 200 -
